In [2]:

import numpy as np
import matplotlib
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, datasets


In [3]:
# Data
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()
num_samples = 50000
train_images = train_images.astype('float32')[:num_samples]
test_images = test_images.astype('float32')
train_images /= 255.0
test_images /= 255.0
train_labels = keras.utils.to_categorical(train_labels, 10)[:num_samples]
test_labels = keras.utils.to_categorical(test_labels, 10)

# Model

# Note: Data augmentation is inactive at test time so input images will only be 
#augmented during calls to Model.fit (not Model.evaluate or Model.predict).

data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.2),
    layers.RandomTranslation(height_factor=0.1, width_factor=0.1, fill_mode='reflect'),
    layers.GaussianNoise(stddev=0.2)
])

input_tensor = layers.Input(shape=(32, 32, 3))

x = data_augmentation(input_tensor)

x = layers.Conv2D(filters=32, kernel_size=(4, 4), activation='relu')(x)
x = layers.MaxPool2D(pool_size=(2, 2), padding='same')(x)
x = layers.Conv2D(filters=32, kernel_size=(4, 4), activation='relu')(x)
x = layers.MaxPool2D(pool_size=(2, 2), padding='same')(x)
x = layers.Conv2D(filters=64, kernel_size=(4, 4), activation='relu')(x)
x = layers.Flatten()(x)
output_tensor = layers.Dense(units=10, activation='softmax')(x)

model = keras.Model(inputs=[input_tensor], outputs=[output_tensor])

# Loss, Optimizer, Compile

loss_fun = keras.losses.CategoricalCrossentropy()

model.compile(loss=loss_fun, optimizer='rmsprop', metrics=['accuracy'])  # sgd, adam, rmsprop

'''
a function that takes an epoch index (integer, indexed from 0) and current learning rate (float) as inputs and returns a new learning rate as output (float).
'''
def scheduler(epoch, lr):
    # epochs:100 0.1 ---> 0.0001...
    lr_params = {
        'total_sample_num': 50_000,
        'epochs': 100,
        'batch_size': 128,
        'initial_lr': 0.1,
        'decay_steps': 120 * 2.5,
        'decay_rate': 0.95
    }
    cur_step = int(lr_params['total_sample_num'] / lr_params['batch_size']) * epoch
    initial_lr = lr_params['initial_lr']
    decay_steps = lr_params['decay_steps']
    decay_rate = lr_params['decay_rate']
    p = cur_step / decay_steps
    return initial_lr * decay_rate ** p

callback = keras.callbacks.LearningRateScheduler(scheduler)

# Training
history = model.fit(train_images,
                    train_labels,
                    validation_data=(test_images, test_labels),
                    epochs=5,
                    batch_size=128,
                    callbacks=[callback],
                    )

170498071/170498071 [==============================] - 10s 0us/step


Epoch 1/5


391/391 [==============================] - 90s 221ms/step - loss: 744.3610 - accuracy: 0.1006 - val_loss: 2.3137 - val_accuracy: 0.1000 - lr: 0.1000
Epoch 2/5
391/391 [==============================] - 89s 228ms/step - loss: 2.3093 - accuracy: 0.0998 - val_loss: 2.3085 - val_accuracy: 0.1000 - lr: 0.0935
Epoch 3/5
391/391 [==============================] - 87s 223ms/step - loss: 2.3093 - accuracy: 0.0980 - val_loss: 2.3061 - val_accuracy: 0.1000 - lr: 0.0875
Epoch 4/5
391/391 [==============================] - 88s 225ms/step - loss: 2.3084 - accuracy: 0.0980 - val_loss: 2.3141 - val_accuracy: 0.1000 - lr: 0.0819
Epoch 5/5
391/391 [==============================] - 86s 221ms/step - loss: 2.3079 - accuracy: 0.1005 - val_loss: 2.3082 - val_accuracy: 0.1000 - lr: 0.0766


In [4]:
# Note: Data augmentation is inactive at test time so input images will only be 
#augmented during calls to Model.fit (not Model.evaluate or Model.predict).
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)

313/313 - 4s - loss: 2.3082 - accuracy: 0.1000 - 4s/epoch - 12ms/step
